# Previsão do valor de mercado das casas na Califórnia

Explore o dataset e construa um modelo de regressão capaz de prever o valor de mercado do imobiliário californiano.

* Divida em treino e teste, com 30% dos dados para teste;
* sempre que necessário, use random_state = 12;

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
# Load the data
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)

## Explore the dataset

In [2]:
X.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB


In [4]:
y = data.target
data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [5]:
# Convert the target variable to a DataFrame for a tabular view
y_df = pd.DataFrame(y, columns=["MedianHouseValue"])

# Display the first 5 rows of the target DataFrame
print(y_df.head(5))

   MedianHouseValue
0             4.526
1             3.585
2             3.521
3             3.413
4             3.422


## Split in Test and Train

In [6]:
# defina a variável alvo
target_variable = 'MedianHouseValue'

# importe a função train_test_split e defina X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 12)

## Simple RFR

In [7]:
# importe o modelo de regressão linear
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# para verificar a accuracy do modelo aplique a função .score
rf.score(X_test, y_test)

0.8106949401709217

In [8]:
# Create a DataFrame to compare the predicted and actual values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Display the first 10 rows of the comparison DataFrame
print(comparison_df.head(10))

   Actual  Predicted
0   2.114    2.00176
1   1.952    1.88462
2   2.418    1.96295
3   1.283    1.75200
4   0.708    1.17150
5   2.024    1.62392
6   3.786    2.50496
7   1.185    1.45591
8   1.165    1.23539
9   1.869    1.91872


## Using GridSearch, but with low parameter, it has 20k data

In [39]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# defina o cv com 5 folds
kf = KFold(n_splits=5)

# crie o modelo de random forest
rf = RandomForestRegressor(random_state=42)

# Define a smaller grid of hyperparameters
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10],
    'min_samples_split': [2]
}

# crie o dict de métricas (rmse, r2)
scoring = {
    'rmse': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

In [40]:

# aplique a grid search ao modelo de Random Forest (faça fit para o rmse)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=kf, scoring=scoring, refit='rmse')
grid_search_rf.fit(X_train, y_train)

# obtenha os melhores hyperparameters
best_params_rf = grid_search_rf.best_params_
print("Best Parameters for Random Forest Regressor:", best_params_rf)

# obtenha o melhor score
best_score_rf = grid_search_rf.best_score_
print("Best Cross-Validation RMSE Score for Random Forest Regressor:", -best_score_rf)

# avalie os resultados para os diferentes k_folds
pd.DataFrame(grid_search_rf.cv_results_)

# obtenha o melhor modelo
best_rf = grid_search_rf.best_estimator_

Best Parameters for Random Forest Regressor: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation RMSE Score for Random Forest Regressor: 0.5164364609311021


In [41]:
# faça as previsões
y_pred_rf = best_rf.predict(X_test)

# avalie as métricas finais
print("\nRandom Forest Regressor Evaluation")
print("Root Mean Squared Error:", mean_squared_error(y_test, y_pred_rf)**(1/2))
print("R2 Score:", r2_score(y_test, y_pred_rf))


Random Forest Regressor Evaluation
Root Mean Squared Error: 0.5037301581279156
R2 Score: 0.8105948808287605
